## TRADE DATA

In [27]:
import pandas as pd
import datetime
import numpy as np

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_wheat_0 = process_csv("trade_data/wheat.csv", 2)
df_wheat_0['OBS_VALUE_QUANTITY_TON'] = df_wheat_0['OBS_VALUE_QUANTITY_TON'].replace(np.nan, 0)
df_wheat_0['OBS_VALUE_VALUE_1000EURO'] = df_wheat_0['OBS_VALUE_VALUE_1000EURO'].replace(np.nan, 0)
df_wheat_0['price'] = df_wheat_0['OBS_VALUE_VALUE_1000EURO'] / df_wheat_0['OBS_VALUE_QUANTITY_TON']


df_wheat_1 = process_csv("trade_data/wheat.csv", 1)
df_wheat_1['OBS_VALUE_QUANTITY_TON'] = df_wheat_1['OBS_VALUE_QUANTITY_TON'].replace(np.nan, 0)
df_wheat_1['OBS_VALUE_VALUE_1000EURO'] = df_wheat_1['OBS_VALUE_VALUE_1000EURO'].replace(np.nan, 0)
df_wheat_1['price'] = df_wheat_1['OBS_VALUE_VALUE_1000EURO'] / df_wheat_1['OBS_VALUE_QUANTITY_TON']

df_wheat_2 = process_csv("trade_data/wheat.csv", 0)
df_wheat_2['OBS_VALUE_QUANTITY_TON'] = df_wheat_2['OBS_VALUE_QUANTITY_TON'].replace(np.nan, 0)
df_wheat_2['OBS_VALUE_VALUE_1000EURO'] = df_wheat_2['OBS_VALUE_VALUE_1000EURO'].replace(np.nan, 0)
df_wheat_2['price'] = df_wheat_2['OBS_VALUE_VALUE_1000EURO'] / df_wheat_2['OBS_VALUE_QUANTITY_TON']

df_wheat01 = df_wheat_0.merge(df_wheat_1, on=["PARTNER_Labels", "TIME_PERIOD"], how='outer')
df_wheat01['OBS_VALUE_QUANTITY_TON_x'] = df_wheat01['OBS_VALUE_QUANTITY_TON_x'].replace(np.nan, 0)
df_wheat01['OBS_VALUE_QUANTITY_TON_y'] = df_wheat01['OBS_VALUE_QUANTITY_TON_y'].replace(np.nan, 0)
df_wheat01['OBS_VALUE_VALUE_1000EURO_x'] = df_wheat01['OBS_VALUE_VALUE_1000EURO_x'].replace(np.nan, 0)
df_wheat01['OBS_VALUE_VALUE_1000EURO_y'] = df_wheat01['OBS_VALUE_VALUE_1000EURO_y'].replace(np.nan, 0)

df_wheat012 = df_wheat01.merge(df_wheat_2, on=["PARTNER_Labels", "TIME_PERIOD"], how='outer')
df_wheat012['OBS_VALUE_QUANTITY_TON'] = df_wheat012['OBS_VALUE_QUANTITY_TON'].replace(np.nan, 0)
df_wheat012['OBS_VALUE_VALUE_1000EURO'] = df_wheat012['OBS_VALUE_VALUE_1000EURO'].replace(np.nan, 0)

df_wheat = df_wheat012.rename(columns={'OBS_VALUE_QUANTITY_TON': 'OBS_VALUE_QUANTITY_TON_z', 'OBS_VALUE_VALUE_1000EURO': 'OBS_VALUE_VALUE_1000EURO_z', 'price': 'price_z'})
df_wheat['OBS_VALUE_QUANTITY_TON'] = df_wheat['OBS_VALUE_QUANTITY_TON_x'] + df_wheat['OBS_VALUE_QUANTITY_TON_y'] + df_wheat['OBS_VALUE_QUANTITY_TON_z']
df_wheat['OBS_VALUE_VALUE_1000EURO'] = df_wheat['OBS_VALUE_VALUE_1000EURO_x'] + df_wheat['OBS_VALUE_VALUE_1000EURO_y'] + df_wheat['OBS_VALUE_VALUE_1000EURO_z']

df_wheat['price'] = df_wheat['OBS_VALUE_VALUE_1000EURO']/df_wheat['OBS_VALUE_QUANTITY_TON']



In [76]:
df_wheat.to_excel("okeeeebijzonder.xlsx")

In [34]:
# group by TIME_PERIOD and aggregate the OBS_VALUE_QUANTITY_TON and OBS_VALUE_VALUE_1000EURO columns by summing
grouped_df = df_wheat.groupby('TIME_PERIOD').agg({'OBS_VALUE_QUANTITY_TON': 'sum', 'OBS_VALUE_VALUE_1000EURO': 'sum'})

# calculate globalprice
globalprice = pd.DataFrame(grouped_df['OBS_VALUE_VALUE_1000EURO'] / grouped_df['OBS_VALUE_QUANTITY_TON'])
# rename the column to 'globalprice'
globalprice = globalprice.rename(columns={0: 'globalprice'})


globalprice = globalprice.reset_index()
globalprice = globalprice.rename(columns={'index': 'TIME_PERIOD'})

globalprice

,TIME_PERIOD,globalprice
0,2005-01-01,0.236286
1,2005-02-01,0.283810
2,2005-03-01,0.221954
3,2005-04-01,0.243158
4,2005-05-01,0.271598
...,...,...
209,2022-06-01,0.337757
210,2022-07-01,0.350521
211,2022-08-01,0.345824
212,2022-09-01,0.324785


In [ ]:
""""
import matplotlib.pyplot as plt

# slice the globalprice dataframe to only include rows from 2012 onwards
globalprice_from_2012 = globalprice[globalprice['TIME_PERIOD'] >= '2012-01-01']

# create a figure and axis object
fig, ax = plt.subplots()

# plot the sliced data
ax.plot(globalprice_from_2012['TIME_PERIOD'], globalprice_from_2012['globalprice'])

# set x-axis label
ax.set_xlabel('Time Period')

# set y-axis label
ax.set_ylabel('Global Price')

# set title
ax.set_title('Global Price Over Time (starting from 2012)')

# show the plot
plt.show()


## WEATHER DATA

In [43]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature['TIME_PERIOD']

0       2005-01-01
1       2005-02-01
2       2005-03-01
3       2005-04-01
4       2005-05-01
           ...    
21085   2022-08-01
21086   2022-09-01
21087   2022-10-01
21088   2022-11-01
21089   2022-12-01
Name: TIME_PERIOD, Length: 21090, dtype: datetime64[ns]

## OIL DATA

In [44]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price['TIME_PERIOD']

540   2005-01-01
541   2005-02-01
542   2005-03-01
543   2005-04-01
544   2005-05-01
         ...    
749   2022-06-01
750   2022-07-01
751   2022-08-01
752   2022-09-01
753   2022-10-01
Name: TIME_PERIOD, Length: 214, dtype: datetime64[ns]

## FUTURES PRICES (WHEAT)

In [48]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_futures_wheat = pd.read_csv('futures_data/US Wheat Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_wheat.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Wheat Price Futures'}, inplace=True)

df_futures_wheat['TIME_PERIOD'] = pd.to_datetime(df_futures_wheat['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_wheat['TIME_PERIOD'] = df_futures_wheat['TIME_PERIOD'].dt.strftime('%Y/%d/%m')

df_futures_wheat.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_wheat['TIME_PERIOD'] = pd.to_datetime(df_futures_wheat['TIME_PERIOD'])

df_futures_wheat.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)
df_futures_wheat['TIME_PERIOD']

213   2005-01-01
212   2005-02-01
211   2005-03-01
210   2005-04-01
209   2005-05-01
         ...    
4     2022-06-01
3     2022-07-01
2     2022-08-01
1     2022-09-01
0     2022-10-01
Name: TIME_PERIOD, Length: 214, dtype: datetime64[ns]

## MACROECONOMIC DATA

In [46]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


In [47]:
import pandas as pd
import datetime
import numpy as np
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

#delete the last 5 empty rows 
num_rows = len(population_data)
population_data = population_data.drop(population_data.index[num_rows-5:num_rows])
#rename a column 
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels','Time':'TIME_PERIOD'})
population_data = population_data.rename(columns = {'Population, total [SP.POP.TOTL]': 'A'})

#get's the index s of the columns with string
df = population_data
string_indices = df['A'][df['A'].apply(lambda x: isinstance(x, str))].index

#gets th countries corresponging to the index s 
lll = len(string_indices)
LIST_of_del_countries = [None] * lll

counter = 0 
for i in range(lll):
    LIST_of_del_countries[counter] = population_data.iloc[(string_indices[i]),0]
    counter = counter +1

list1 = list(set(LIST_of_del_countries))

print(df.shape)
print(list1)
for j in range(4000):
  
    if df.iloc[j,0] in list1:
        df = df.drop(index=[j])
        print(df.shape)

df = df.replace('..', 0)
print(df)
# boolean indexing to select rows to delete
idx = df['PARTNER_Labels'].str.contains('IDA blend')

# drop rows using boolean indexing
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA only')
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA total')
df = df.drop(df[idx].index)

for name in list1:
    idx = df['PARTNER_Labels'].str.contains(name)
    df = df.drop(df[idx].index)


testing= df.copy()
excel_test = pd.ExcelWriter("population_data.xlsx")
testing.to_excel(excel_test)
excel_test.save()
#make the 
monthly_data_p = population_data.iloc[0:0].copy()

number_of_col = df.shape[1]
nan_row = np.empty((number_of_col,))
nan_row[:] = np.nan

n_row = df.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = df.iloc[i,:]
    current_country = df.iloc[i,0]
    current_country_code = df.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data_p.loc[index_3] = holder_for_rows   
        else:
            monthly_data_p.loc[index_3] = nan_row
            monthly_data_p.iloc[index_3,0] = current_country
            monthly_data_p.iloc[index_3,1] = current_country_code 


# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper2.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()

c_p = population_data.columns[4]
monthly_data_p[c_p] = monthly_data_p[c_p].apply(pd.to_numeric)
n_countries = len(df['Country Code'].unique())
#fills in the date column
for i in range(n_countries-10):
    index_1 = 0   + 228*i
    index_2 = 228 + 228*i
    monthly_data_p.iloc[index_1:index_2,4] = monthly_data_p.iloc[index_1:index_2,4].interpolate().copy()
    #adding dates 
    monthly_data_p.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2023-12", freq='M')

# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper3.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()


(5054, 5)
['IDA total', 'Not classified', 'Sint Maarten (Dutch part)', 'IDA blend', 'Bermuda', 'St. Martin (French part)', 'IDA only', 'Greenland', 'IBRD only']
(5053, 5)
(5052, 5)
(5051, 5)
(5050, 5)
(5049, 5)
(5048, 5)
(5047, 5)
(5046, 5)
(5045, 5)
(5044, 5)
(5043, 5)
(5042, 5)
(5041, 5)
(5040, 5)
(5039, 5)
(5038, 5)
(5037, 5)
(5036, 5)
(5035, 5)
(5034, 5)
(5033, 5)
(5032, 5)
(5031, 5)
(5030, 5)
(5029, 5)
(5028, 5)
(5027, 5)
(5026, 5)
(5025, 5)
(5024, 5)
(5023, 5)
(5022, 5)
(5021, 5)
(5020, 5)
(5019, 5)
(5018, 5)
(5017, 5)
(5016, 5)
(5015, 5)
(5014, 5)
(5013, 5)
(5012, 5)
(5011, 5)
(5010, 5)
(5009, 5)
(5008, 5)
(5007, 5)
(5006, 5)
(5005, 5)
(5004, 5)
(5003, 5)
(5002, 5)
(5001, 5)
(5000, 5)
(4999, 5)
(4998, 5)
(4997, 5)
(4996, 5)
(4995, 5)
(4994, 5)
(4993, 5)
(4992, 5)
(4991, 5)
(4990, 5)
(4989, 5)
(4988, 5)
(4987, 5)
(4986, 5)
(4985, 5)
(4984, 5)
(4983, 5)
(4982, 5)
(4981, 5)
(4980, 5)
(4979, 5)
(4978, 5)
(4977, 5)
(4976, 5)
(4975, 5)
     PARTNER_Labels Country Code  TIME_PERIOD Tim

C:\Users\basbe\AppData\Local\Temp\ipykernel_21408\3099761979.py:54: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df['PARTNER_Labels'].str.contains(name)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705


In [49]:
monthly_data_p  = monthly_data_p.rename(columns = {'Country Code': 'country_code'})
# convert Period and float values to string format, and then to datetime
monthly_data_p ['TIME_PERIOD'] = monthly_data_p ['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data_p ['TIME_PERIOD'] = pd.to_datetime(monthly_data_p ['TIME_PERIOD'], errors='coerce')
monthly_data_p.head(214)

,PARTNER_Labels,country_code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06-01,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07-01,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08-01,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09-01,NaN,4.186949e+07


In [50]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

In [51]:
merged_monthly_data_df = monthly_data.merge(monthly_data_p, on=['country_code', 'TIME_PERIOD'], how='outer')
merged_monthly_data_df = merged_monthly_data_df.drop('PARTNER_Labels_y', axis=1)
merged_monthly_data_df = merged_monthly_data_df.rename(columns={'PARTNER_Labels_x': 'PARTNER_Labels', 'A': 'population_projections'})


In [52]:
df_macroeconomic = merged_monthly_data_df
df_macroeconomic

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,Argentina,ARG,2005-01-01 00:00:00.000000000,1.987371e+11,4.352966e+11,39070501.0,11.510000,1.299209e+06,9.640000,1712.850000,89.050000,YR2005,39070501.0
1,Argentina,ARG,2005-02-01 00:00:00.000000000,2.015554e+11,4.382157e+11,39104363.5,11.390833,1.299391e+06,9.601667,1723.801667,89.016667,NaN,39104363.5
2,Argentina,ARG,2005-03-01 00:00:00.000000000,2.043738e+11,4.411348e+11,39138226.0,11.271667,1.299572e+06,9.563333,1734.753333,88.983333,NaN,39138226.0
3,Argentina,ARG,2005-04-01 00:00:00.000000000,2.071921e+11,4.440538e+11,39172088.5,11.152500,1.299754e+06,9.525000,1745.705000,88.950000,NaN,39172088.5
4,Argentina,ARG,2005-05-01 00:00:00.000000000,2.100105e+11,4.469729e+11,39205951.0,11.033333,1.299936e+06,9.486667,1756.656667,88.916667,NaN,39205951.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61747,NaN,ZWE,1970-01-01 00:00:00.000002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2019,15354608.0
61748,NaN,ZWE,1970-01-01 00:00:00.000002020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2020,15669666.0
61749,NaN,ZWE,1970-01-01 00:00:00.000002021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2021,15993524.0
61750,NaN,ZWE,1970-01-01 00:00:00.000002022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2022,16320537.0


## MERGING DATA

### Start with empty dataframe with all possible combinations

In [54]:
import itertools
import pandas as pd

df_wheat['TIME_PERIOD'] = pd.to_datetime(df_wheat['TIME_PERIOD'])

# Get all unique values for TIME_PERIOD and PARTNER_Labels
time_periods = df_wheat['TIME_PERIOD'].unique()
partner_labels = df_wheat['PARTNER_Labels'].unique()

# Generate all possible combinations
combinations = list(itertools.product(time_periods, partner_labels))

# Create a new dataframe with the combinations
df_combinations = pd.DataFrame(combinations, columns=['TIME_PERIOD', 'PARTNER_Labels'])

# Merge with the original dataframe to get the corresponding values
df_result = pd.merge(df_combinations, df_wheat, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')

# Fill any missing values with 0
df_result = df_result.fillna(0)
df_result.head(200)

,TIME_PERIOD,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,product,OBS_VALUE_QUANTITY_TON_z,OBS_VALUE_VALUE_1000EURO_z,price_z,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,2010-03-01,Austria,100810.0,25.7,7.95,0.309339,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,25.7,7.95,0.309339
1,2010-03-01,Belarus,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
2,2010-03-01,Belgium,100810.0,0.6,0.45,0.750000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.6,0.45,0.750000
3,2010-03-01,Brazil,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
4,2010-03-01,Bulgaria,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2016-05-01,Czechia,100810.0,0.0,0.02,inf,100199.0,0.2,0.07,0.350000,100119.0,1.3,0.37,0.284615,1.5,0.46,0.306667
196,2016-05-01,Denmark,100810.0,0.2,0.38,1.900000,100199.0,12.7,4.07,0.320472,100119.0,1.0,0.29,0.290000,13.9,4.74,0.341007
197,2016-05-01,Estonia,100810.0,0.7,1.20,1.714286,100199.0,1.0,0.32,0.320000,100119.0,0.2,0.05,0.250000,1.9,1.57,0.826316
198,2016-05-01,Finland,100810.0,0.0,0.04,inf,100199.0,3.6,1.15,0.319444,100119.0,1.2,0.33,0.275000,4.8,1.52,0.316667


## Merge with global price data

In [55]:
df_result['TIME_PERIOD'] = pd.to_datetime(df_result['TIME_PERIOD'])
globalprice['TIME_PERIOD'] = pd.to_datetime(globalprice['TIME_PERIOD'])

# add globalprice as a new column to the dataframe
df_merged_result_globalprice = df_result.merge(globalprice, on=['TIME_PERIOD'], how='right')
df_merged_result_globalprice = df_merged_result_globalprice.rename(columns={"globalprice_y": "globalprice"})

df_merged_result_globalprice

,TIME_PERIOD,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,product,OBS_VALUE_QUANTITY_TON_z,OBS_VALUE_VALUE_1000EURO_z,price_z,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,Austria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
1,2005-01-01,Belarus,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
2,2005-01-01,Belgium,100810.0,0.4,1.09,2.725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.09,2.725,0.236286
3,2005-01-01,Brazil,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
4,2005-01-01,Bulgaria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,2022-10-01,Syrian Arab Republic,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13264,2022-10-01,Taiwan,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13265,2022-10-01,United Arab Emirates,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13266,2022-10-01,Morocco,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141


### Merged trade data, oil price

In [56]:
df_merged_trade_oil = df_oil_price.merge(df_merged_result_globalprice, on='TIME_PERIOD', how='outer')
df_merged_trade_oil

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,product,OBS_VALUE_QUANTITY_TON_z,OBS_VALUE_VALUE_1000EURO_z,price_z,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
1,2005-01-01,42.972278,Belarus,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.09,2.725,0.236286
3,2005-01-01,42.972278,Brazil,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
4,2005-01-01,42.972278,Bulgaria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.236286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,2022-10-01,90.326667,Syrian Arab Republic,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13264,2022-10-01,90.326667,Taiwan,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13265,2022-10-01,90.326667,United Arab Emirates,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141
13266,2022-10-01,90.326667,Morocco,0.0,0.0,0.00,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.338141


### Merged trade data, oil price, weather data

In [57]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,OBS_VALUE_QUANTITY_TON_z,OBS_VALUE_VALUE_1000EURO_z,price_z,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.000,0.236286,-1.166667,-172.500000,125.000000
1,2005-01-01,42.972278,Belarus,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.000,0.236286,-7.788025,-132.900000,90.437500
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,0.0,0.0,0.0,...,0.0,0.0,0.0,0.4,1.09,2.725,0.236286,30.700000,-31.000000,79.000000
3,2005-01-01,42.972278,Brazil,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.000,0.236286,269.701032,182.000000,355.500000
4,2005-01-01,42.972278,Bulgaria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.000,0.236286,24.086022,-78.000000,172.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22594,2022-08-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.060676,244.363636,348.666667
22595,2022-09-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279.751352,233.600000,350.000000
22596,2022-10-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.000000,238.000000,260.000000
22597,2022-11-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.000000,252.000000,304.000000


### Merged trade data, oil price, weather, future prices data

In [58]:
df_merged_trade_oil_weather['TIME_PERIOD'] = pd.to_datetime(df_merged_trade_oil_weather['TIME_PERIOD'] , format='%Y/%m/%d')

df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_wheat, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures.head(4000)

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,OBS_VALUE_VALUE_1000EURO_z,price_z,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Wheat Price Futures
0,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.236286,-1.166667,-172.500000,125.000000,291.00
1,2005-01-01,42.972278,Belarus,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.236286,-7.788025,-132.900000,90.437500,291.00
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,0.0,0.0,0.0,...,0.0,0.0,0.4,1.09,2.725,0.236286,30.700000,-31.000000,79.000000,291.00
3,2005-01-01,42.972278,Brazil,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.236286,269.701032,182.000000,355.500000,291.00
4,2005-01-01,42.972278,Bulgaria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.236286,24.086022,-78.000000,172.500000,291.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2008-03-01,101.842833,Hungary,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.347819,65.378495,-34.200000,169.800000,929.00
3996,2008-03-01,101.842833,India,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.347819,262.341338,160.402985,348.000000,929.00
3997,2008-03-01,101.842833,"Iran, Islamic Republic of",0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.347819,160.437442,19.590909,291.384615,929.00
3998,2008-03-01,101.842833,Ireland,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000,0.347819,65.852880,-7.571429,119.200000,929.00


### Merged trade, oil price, weather, future prices, macroeconomic 

In [59]:
df_macroeconomic['TIME_PERIOD'] = pd.to_datetime(df_macroeconomic['TIME_PERIOD'] , format='%Y/%m/%d')

df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged



,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Austria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
1,2005-01-01,42.972278,Belarus,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,3.020757e+10,3.749613e+10,9663915.0,1.50,89490.0,6.83,2768.82,92.67,YR2005,9663915.0
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,0.0,0.0,0.0,...,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
3,2005-01-01,42.972278,Brazil,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,8.916302e+11,1.368461e+12,186797334.0,10.55,2288420.0,46.13,1152.76,54.25,YR2005,186797334.0
4,2005-01-01,42.972278,Bulgaria,0.0,0.0,0.00,0.000,0.0,0.0,0.0,...,2.986928e+10,4.006637e+10,7658972.0,10.08,52650.0,9.83,2597.90,72.96,YR2005,7658972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22594,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22595,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22596,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22597,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CLEANING EXTREME VALUES

In [60]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)
df_merged = df_merged.replace(0, np.nan)
df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
1,2005-01-01,42.972278,Belarus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.020757e+10,3.749613e+10,9663915.0,1.50,89490.0,6.83,2768.82,92.67,YR2005,9663915.0
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,NaN,NaN,NaN,...,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
3,2005-01-01,42.972278,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.916302e+11,1.368461e+12,186797334.0,10.55,2288420.0,46.13,1152.76,54.25,YR2005,186797334.0
4,2005-01-01,42.972278,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.986928e+10,4.006637e+10,7658972.0,10.08,52650.0,9.83,2597.90,72.96,YR2005,7658972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22594,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22595,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22596,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22597,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total', 'unemployment_total', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)

df_merged


,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01,42.972278,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
1,2005-01-01,42.972278,Belarus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.020757e+10,3.749613e+10,9663915.0,1.50,89490.0,6.83,2768.82,92.67,YR2005,9663915.0
2,2005-01-01,42.972278,Belgium,100810.0,0.4,1.09,2.725,NaN,NaN,NaN,...,3.857148e+11,4.026519e+11,10478617.0,8.44,13830.0,2.46,5555.62,74.76,YR2005,10478617.0
3,2005-01-01,42.972278,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.916302e+11,1.368461e+12,186797334.0,10.55,2288420.0,46.13,1152.76,54.25,YR2005,186797334.0
4,2005-01-01,42.972278,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.986928e+10,4.006637e+10,7658972.0,10.08,52650.0,9.83,2597.90,72.96,YR2005,7658972.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22594,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22595,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22596,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22597,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## FULL DATAFRAME (without filtering top countries)

In [64]:
pd.set_option('display.max_columns', 2000) #replace n with the number of columns you want to see completely
###FOR NOW
df_wheat_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['globalprice', 'OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Wheat Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=False)
df_wheat_total['price'].head(100)

PARTNER_Labels,Argentina,Austria,Belarus,Belgium,Bulgaria,Canada,China,Croatia,Cyprus,Czechia,Denmark,Egypt,Estonia,Finland,France,Germany,Greece,Hungary,India,"Iran, Islamic Republic of",Iraq,Ireland,Israel,Italy,Japan,Jordan,Kazakhstan,Latvia,Lithuania,Luxembourg,Malta,Moldova,New Zealand,Nigeria,Occupied palestinian Territory,Peru,Poland,Portugal,Romania,Russia,Serbia,Slovakia,Slovenia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Türkiye,Ukraine,United Arab Emirates,United Kingdom,United States
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0.220451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-02-01,NaN,NaN,NaN,NaN,NaN,NaN,0.283650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.269098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-03-01,NaN,NaN,NaN,NaN,NaN,NaN,0.215390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.637500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04-01,NaN,NaN,NaN,NaN,NaN,NaN,0.227308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.294965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-05-01,NaN,NaN,NaN,NaN,NaN,NaN,0.268705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.301992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.011538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-01,NaN,NaN,NaN,0.263187,NaN,NaN,0.866900,NaN,NaN,NaN,0.253139,NaN,NaN,NaN,0.248770,0.258784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400638,0.246883,NaN,NaN,NaN,NaN,NaN,NaN,0.392961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.29397,NaN,0.207642,NaN
2013-01-01,NaN,NaN,NaN,0.266810,NaN,NaN,0.724203,NaN,NaN,NaN,0.249246,NaN,NaN,NaN,0.253235,0.268869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.236208,NaN,NaN,0.616567,NaN,NaN,0.2,0.383767,NaN,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.278658,NaN,NaN,NaN,NaN,NaN,NaN,0.190681,NaN
2013-02-01,NaN,NaN,NaN,0.271667,NaN,NaN,0.536364,NaN,NaN,NaN,0.253112,NaN,NaN,NaN,0.253197,0.263031,NaN,0.252996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325474,0.235472,NaN,NaN,NaN,NaN,NaN,NaN,0.504078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.281771,NaN,NaN,NaN,NaN,NaN,NaN,0.508824,NaN


## HANDLE MISSING DATA

### Fill in missing values with global price

In [65]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_wheat_total[['price']].loc[:, ('price', df_wheat_total['price'].isna().any())]
df_missing_price = df_missing_price.iloc[:-2]
# Forward fill any remaining missing values
df_missing_price['price'] = df_missing_price['price'].fillna(method='ffill')
# Back fill any missing values at the beginning of the series
df_missing_price['price'] = df_missing_price['price'].fillna(method='bfill')


In [68]:
###FOR NOW
#df_sunflower_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=True)

# Create a new DataFrame that only contains the globalprice
df_price = globalprice['globalprice']
df_price = pd.DataFrame(df_price)
df_price.set_index(globalprice['TIME_PERIOD'], inplace=True)

# Calculate the year-over-year percent change in 'price' for France and Germany:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)

df_price_pct_change = pd.DataFrame(df_price_pct_change)
df_price_pct_change

,globalprice
TIME_PERIOD,
2005-01-01,0.000000
2005-02-01,0.201129
2005-03-01,-0.217949
2005-04-01,0.095534
2005-05-01,0.116960
...,...
2022-06-01,0.078301
2022-07-01,0.037790
2022-08-01,-0.013399


In [70]:
multiplyfactor = 1 + df_price_pct_change['globalprice']
multiplyfactor = pd.DataFrame(multiplyfactor)

df_result1 = df_missing_price.multiply(multiplyfactor['globalprice'], axis='index')

mask = df_wheat_total['price'].isna()

df_wheat_total_updated = df_wheat_total.combine_first(df_result1[mask])
df_wheat_total_updated['price']

PARTNER_Labels,Argentina,Austria,Belarus,Belgium,Bulgaria,Canada,China,Croatia,Cyprus,Czechia,Denmark,Egypt,Estonia,Finland,France,Germany,Greece,Hungary,India,"Iran, Islamic Republic of",Iraq,Ireland,Israel,Italy,Japan,Jordan,Kazakhstan,Latvia,Lithuania,Luxembourg,Malta,Moldova,New Zealand,Nigeria,Occupied palestinian Territory,Peru,Poland,Portugal,Romania,Russia,Serbia,Slovakia,Slovenia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Türkiye,Ukraine,United Arab Emirates,United Kingdom,United States
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.350000,0.309339,0.600000,0.696667,0.160500,0.455037,0.220451,0.159398,0.958621,0.338025,0.186941,0.303333,0.244441,0.243000,0.566000,2.040000,0.338095,0.194994,0.723037,0.450000,0.436000,0.300000,0.181818,0.339841,0.640000,0.195906,0.322205,0.180397,0.306977,0.211480,0.312500,0.310003,0.587451,0.200000,0.180597,0.200000,0.268833,0.880000,0.245005,1.720000,0.178553,0.745255,0.336842,0.200000,1.213710,0.150000,0.189423,1.622222,0.560000,0.233333,0.219979,0.304500,0.200000,0.388095,1.011538
2005-02-01,0.420395,0.371555,0.720677,0.836786,0.192781,0.546558,0.283650,0.191457,1.151427,0.406012,0.224540,0.364342,0.293605,0.291874,0.679839,0.600000,0.406096,0.234213,0.868460,0.450000,0.523692,0.360339,0.218387,0.408192,0.768722,0.235309,0.387010,0.216681,0.368719,0.254015,0.375353,0.372353,0.705604,0.240226,0.216920,0.240226,0.269098,1.056993,0.294283,2.065941,0.214465,0.895148,0.404591,0.240226,1.457821,0.180169,0.227521,1.948498,0.672632,0.280263,0.264223,0.365744,0.240226,0.466152,1.214988
2005-03-01,0.273718,0.241919,0.469231,0.544829,0.125519,0.355862,0.215390,0.124657,0.749691,0.264353,0.146197,0.237222,0.191165,0.190038,0.442641,0.637500,0.264408,0.152495,0.565452,0.351923,0.340974,0.234615,0.142191,0.265773,0.500513,0.153209,0.251981,0.141080,0.240072,0.165388,0.244391,0.242438,0.459417,0.156410,0.141236,0.156410,0.276785,0.688205,0.191607,1.345128,0.139638,0.582828,0.263428,0.156410,0.949183,0.117308,0.148139,1.268661,0.437949,0.182479,0.172035,0.238135,0.156410,0.303510,0.791075
2005-04-01,0.383437,0.338891,0.657320,0.763222,0.175833,0.498509,0.227308,0.174625,1.050202,0.370318,0.204800,0.332312,0.267793,0.266214,0.620072,0.275934,0.370395,0.213623,0.792111,0.492990,0.477653,0.328660,0.199188,0.372307,0.701142,0.214622,0.352987,0.197632,0.336303,0.231684,0.342354,0.339618,0.643573,0.219107,0.197850,0.219107,0.294965,0.964070,0.268411,1.884319,0.195611,0.816453,0.369022,0.219107,1.329660,0.164330,0.207519,1.777200,0.613499,0.255625,0.240994,0.333590,0.219107,0.425172,1.108175
2005-05-01,0.390936,0.345519,0.670176,0.778149,0.179272,0.508258,0.268705,0.178041,1.070741,0.377561,0.208805,0.338811,0.273030,0.271421,0.632199,0.308207,0.377639,0.217800,0.807603,0.502632,0.486994,0.335088,0.203084,0.379588,0.714854,0.218820,0.359890,0.201497,0.342881,0.236215,0.349050,0.346261,0.656159,0.223392,0.201720,0.223392,0.301992,0.982925,0.273661,1.921171,0.199437,0.832420,0.376239,0.223392,1.355665,0.167544,0.211578,1.811957,0.625498,0.260624,0.245707,0.340114,0.223392,0.433487,1.011538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,0.214003,0.200648,0.589220,0.340014,0.203937,0.200000,1.591856,0.203774,0.100000,0.333764,0.202067,0.299269,0.201923,0.205385,0.363200,0.330081,0.203670,0.203520,0.276248,1.430572,0.418639,0.201499,0.179382,0.209049,0.169132,0.203218,1.370279,0.204167,0.234704,0.310469,0.100000,0.779847,0.197320,0.887941,0.217733,0.197320,0.288278,0.213457,0.203090,0.791075,0.198811,0.203922,0.202064,0.197320,0.203094,0.177588,0.207203,1.654201,0.552497,0.230207,1.735257,0.430780,0.197320,0.322478,0.197320
2022-09-01,0.203713,0.323624,0.560889,0.311740,0.212160,0.187833,1.756667,0.221429,0.200000,0.364544,0.203605,0.284880,0.215267,0.213627,0.

### Interpolate macroeconomic

In [71]:
# fill missing values at the beginning of each column with the first known value
df_wheat_total_updated.fillna(method='ffill')

# fill missing values at the end of each column with the last known value
df_wheat_total_updated.fillna(method='bfill')

# interpolate missing values for selected columns
cols_to_interpolate = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

for col in cols_to_interpolate:
    df_wheat_total_updated.loc[:, (col, slice(None))] = df_wheat_total.loc[:, (col, slice(None))].interpolate()
    df_wheat_total_updated.fillna(method='bfill')
    df_wheat_total_updated.fillna(method='ffill')

# print the updated pivot table dataframe
df_wheat_total_updated


AVG_TAVG                                             \
PARTNER_Labels     Albania     Algeria Angola     Armenia   Australia   
TIME_PERIOD                                                             
2005-01-01             NaN   76.228202  274.0  -87.478261  242.290980   
2005-02-01             NaN   84.478184  270.5  -87.000000  237.755531   
2005-03-01             NaN  147.382230  156.0  -46.440000  223.480383   
2005-04-01             NaN  181.541272  242.5   39.714286  208.498275   
2005-05-01             NaN  240.048870  211.5   96.043478  167.861508   
...                    ...         ...    ...         ...         ...   
2022-08-01      144.666667  312.218029  213.0  209.490002  146.200527   
2022-09-01      144.666667  286.008954  213.0  173.108906  165.721311   
2022-10-01      144.666667  231.950455  213.0  111.054056  189.637400   
2022-11-01      144.666667  168.537960  213.0   50.784508  201.174639   
2022-12-01      144.666667  150.914659  213.0   -1.926037  224.700188   

                                                                            \
PARTNER_Labels     Austria  Azerbaijan     Bahrain     Belarus     Belgium   
TIME_PERIOD                                                                  
2005-01-01       -1.166667         NaN  160.222222   -7.788025   30.700000   
2005-02-01      -42.166667   72.000000  176.000000  -59.820327   20.000000   
2005-03-01       -1.994624   68.000000  211.095238  -32.497550   55.800000   
2005-04-01       60.233333  191.000000  257.315789   80.398109  111.000000   
2005-05-01      108.758065  200.000000  303.117647  130.261818  129.250000   
...                    ...         ...         ...         ...         ...   
2022-08-01      175.258849  260.574138  360.153846  207.011045  213.200000   
2022-09-01      103.779082  218.628333  346.350000   99.815254  166.000000   
2022-10-01       98.433871  160.495556  307.000000   82.525416  147.333333   
2022-11-01       36.939583   96.659524  274.357143   17.335752   88.454545   
2022-12-01       -9.296552   37.345238  223.176471  -29.105331   36.900000   

                                                                            \
PARTNER_Labels Bosnia and Herzegovina      Brazil    Bulgaria Burkina Faso   
TIME_PERIOD                                                                  
2005-01-01                 -14.969892  269.701032   24.086022   253.052381   
2005-02-01                 -36.017857  266.743474   -9.425595   302.660095   
2005-03-01                  20.650538  266.528149   54.857527   332.319386   
2005-04-01                  78.601667  253.719444  113.894444   336.259203   
2005-05-01                 134.995161  247.859946  170.809140   315.863580   
...                               ...         ...         ...          ...   
2022-08-01                 218.941935  222.129167  243.446237   260.923571   
2022-09-01                 189.631034  221.270833  180.200000   260.132837   
2022-10-01                 172.066092  242.066667  138.333333   280.302304   
2022-11-01                 117.805769  238.194444   96.594444   271.984864   
2022-12-01                  88.849383  249.907407   49.658602   260.295430   

                                                                            \
PARTNER_Labels    Cameroon      Canada       Chile       China    Colombia   
TIME_PERIOD                                                                  
2005-01-01      265.058824         NaN  193.505177  -39.628008  227.893553   
2005-02-01      301.824074         NaN  194.394702  -25.939147  254.429167   
2005-03-01      328.450000 -104.471986  179.134537   50.029574  258.143750   
2005-04-01      310.826087  -20.064078  152.958690  132.430808  251.740190   
2005-05-01      290.272727   39.783135  138.762942  176.286936  251.447116   
...                    ...         ...         ...         ...         ...   
2022-08-01      231.275000  179.322868   98.776587  234.647872  248.137771   
2022-09-01      224.533333  1

## SAVE EXCEL FILE FULL DATAFRAME

In [72]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'WHEAT_FULL_{now}.xlsx'
df_wheat_total_updated.to_excel(filename, index=True)

## STANDARDIZATION

In [73]:
from sklearn.preprocessing import StandardScaler

df_wheat_total_updated_std = df_wheat_total_updated.copy()

# get a list of all the column names in the DataFrame
all_cols = list(df_wheat_total_updated_std.columns)

# create a list of all columns to normalize, excluding 'price' for all countries
cols_to_normalize = [col for col in all_cols if 'price' not in col]

# instantiate the StandardScaler object
scaler = StandardScaler()

# fit the scaler on the selected columns
scaler.fit(df_wheat_total_updated_std[cols_to_normalize])

# transform the selected columns using the scaler
df_wheat_total_updated_std[cols_to_normalize] = scaler.transform(df_wheat_total_updated_std[cols_to_normalize])
df_wheat_total_updated_std

C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


AVG_TAVG                                                    \
PARTNER_Labels   Albania   Algeria    Angola   Armenia Australia   Austria   
TIME_PERIOD                                                                  
2005-01-01           NaN -1.625629  1.034987 -1.798018  1.252027 -1.083560   
2005-02-01           NaN -1.515783  0.895962 -1.792527  1.137852 -1.677014   
2005-03-01           NaN -0.678237 -3.652156 -1.326795  0.778492 -1.095545   
2005-04-01           NaN -0.223420 -0.216241 -0.337526  0.401334 -0.194828   
2005-05-01           NaN  0.555589 -1.447610  0.309276 -0.621650  0.507542   
...                  ...       ...       ...       ...       ...       ...   
2022-08-01     -0.353314  1.516497 -1.388027  1.611930 -1.166940  1.470106   
2022-09-01     -0.353314  1.167532 -1.388027  1.194183 -0.675527  0.435474   
2022-10-01     -0.353314  0.447761 -1.388027  0.481636 -0.073467  0.358105   
2022-11-01     -0.353314 -0.396555 -1.388027 -0.210411  0.216970 -0.531992   
2022-12-01     -0.353314 -0.631203 -1.388027 -0.815662  0.809199 -1.201236   

                                                         \
PARTNER_Labels Azerbaijan   Bahrain   Belarus   Belgium   
TIME_PERIOD                                               
2005-01-01            NaN -1.872269 -0.977534 -1.407486   
2005-02-01      -1.056052 -1.628760 -1.582130 -1.595379   
2005-03-01      -1.105865 -1.087113 -1.264649 -0.966726   
2005-04-01       0.425891 -0.373761  0.047158  0.002594   
2005-05-01       0.537971  0.333129  0.626557  0.323066   
...                   ...       ...       ...       ...   
2022-08-01       1.292319  1.213405  1.518355  1.797240   
2022-09-01       0.769955  1.000361  0.272778  0.968402   
2022-10-01       0.046010  0.393048  0.071877  0.640612   
2022-11-01      -0.748960 -0.110751 -0.685604 -0.393308   
2022-12-01      -1.487619 -0.900655 -1.225233 -1.298613   

                                                                        \
PARTNER_Labels Bosnia and Herzegovina    Brazil  Bulgaria Burkina Faso   
TIME_PERIOD                                                              
2005-01-01                  -1.828173  1.315208 -1.272341    -1.362267   
2005-02-01                  -2.108647  1.191699 -1.691326     0.632883   
2005-03-01                  -1.353515  1.182707 -0.887616     1.825736   
2005-04-01                  -0.581289  0.647810 -0.149497     1.984190   
2005-05-01                   0.170181  0.403115  0.562088     1.163908   
...                               ...       ...       ...          ...   
2022-08-01                   1.288810 -0.671414  1.470245    -1.045700   
2022-09-01                   0.898229 -0.707258  0.679499    -1.077502   
2022-10-01                   0.664168  0.161185  0.156054    -0.266315   
2022-11-01                  -0.058876 -0.000521 -0.365793    -0.600830   
2022-12-01                  -0.444733  0.488618 -0.952616    -1.070962   

                                                                  \
PARTNER_Labels  Cameroon    Canada     Chile     China  Colombia   
TIME_PERIOD                                                        
2005-01-01     -0.036075       NaN  1.463038 -1.586006 -0.527733   
2005-02-01      1.411027       NaN  1.489391 -1.443774  0.675055   
2005-03-01      2.459040 -1.295559  1.037288 -0.654437  0.843427   
2005-04-01      1.765352 -0.429196  0.261793  0.201736  0.553170   
2005-05-01      0.956359  0.185077 -0.158775  0.657414  0.539886   
...                  ...       ...       ...       ...       ...   
2022-08-01     -1.365827  1.617314 -1.343425  1.263801  0.389883   
2022-09-01     -1.631183  1.122685 -0.648089  0.844932  0.404835   
2022-10-01     -1.720400  0.521893  0.205445  0.062542 -0.086596   
2022-11-01     -1.809618 -0.600848  0.499614 -0.534900 -0.060458   
2022-12-01     -1.809618 -1.290403  1.053763 -1.512325  0.031090   

                                                                              \
PARTNER_Labels Congo,  Dem

## SAVE EXCEL FILE

### STANDARDIZED FULL DATA

In [85]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'WHEAT_FULL_STANDARDIZED_{now}.xlsx'
df_wheat_total_updated_std.to_excel(filename, index=True)

## ANALYSIS WHEAT IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [74]:
total = df_wheat['OBS_VALUE_QUANTITY_TON'].sum()

total

45999082.6

In [75]:
df_filtered = df_wheat

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)   

,index,PARTNER_Labels,OBS_VALUE_QUANTITY_TON,cumulative_sum,cumulative_sum_pct
0,16,France,20246852.6,20246852.6,0.440158
1,17,Germany,12241696.3,32488548.9,0.706287
2,4,Belgium,4811876.1,37300425.0,0.810895
3,60,United Kingdom,1954062.8,39254487.8,0.853375
4,44,Romania,1522441.0,40776928.8,0.886473
5,12,Denmark,731153.6,41508082.4,0.902368
6,31,Latvia,669700.5,42177782.9,0.916927
7,58,Ukraine,595511.6,42773294.5,0.929873
8,33,Lithuania,521294.0,43294588.5,0.941205
9,52,Sweden,494581.1,43789169.6,0.951957


In [79]:
# Declare splitting point for cumulative import to NL history, biggest partners
splitting_point = 0.88

for i, row in grouped.iterrows():
    if row['cumulative_sum_pct'] >= splitting_point:
        top_x = i + 1
        break
        
top_partners = grouped['PARTNER_Labels'].head(top_x).tolist()
top_partners

['France', 'Germany', 'Belgium', 'United Kingdom', 'Romania']

## FILTERING TOP PARTNERS

In [80]:
pd.set_option('display.max_columns', 90) #replace n with the number of columns you want to see completely

### FULL DATA

In [81]:
# filter the pivot table using the loc method
df_wheat_total_filtered = df_wheat_total_updated.loc[:, (slice(None), top_partners)]


# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_wheat_total_filtered = df_wheat_total_filtered.sort_index(axis=1)
df_wheat_total_filtered.head(5)

AVG_TAVG                                                     \
PARTNER_Labels  Belgium      France     Germany     Romania United Kingdom   
TIME_PERIOD                                                                  
2005-01-01        30.70   50.081198   28.651592   -0.185630      60.656665   
2005-02-01        20.00   30.689158   -1.962406  -32.047727      43.345284   
2005-03-01        55.80   76.202630   45.730051   23.051026      65.292148   
2005-04-01       111.00  108.081105  101.859649   92.138788      79.556617   
2005-05-01       129.25  147.068032  134.667233  153.902346      98.090579   

               MAX_TMAX                                                     \
PARTNER_Labels  Belgium      France     Germany     Romania United Kingdom   
TIME_PERIOD                                                                  
2005-01-01         79.0  124.038462  129.842105  109.400000     116.500000   
2005-02-01        100.0  119.983051  108.315789   66.727273      98.312500   
2005-03-01        190.0  201.140625  180.894737  166.727273     140.666667   
2005-04-01        200.0  254.950820  213.315789  205.900000     163.062500   
2005-05-01        223.0  279.101695  314.684211  276.818182     192.466667   

               MIN_TMIN                                                    \
PARTNER_Labels  Belgium     France     Germany     Romania United Kingdom   
TIME_PERIOD                                                                 
2005-01-01        -31.0 -40.377358  -91.000000 -103.000000     -15.153846   
2005-02-01        -66.0 -65.083333 -125.684211 -181.636364     -15.750000   
2005-03-01        -48.0 -90.450000  -99.526316 -142.363636     -18.363636   
2005-04-01         57.0  23.620690  -12.421053  -17.454545      -6.083333   
2005-05-01         53.0  56.158730   22.631579   51.818182       1.466667   

               OBS_VALUE_QUANTITY_TON                                        \
PARTNER_Labels                Belgium France Germany Romania United Kingdom   
TIME_PERIOD                                                                   
2005-01-01                        0.4    NaN     0.5     NaN            NaN   
2005-02-01                        0.4    NaN     1.5     NaN            NaN   
2005-03-01                        0.4    NaN     0.8     NaN            NaN   
2005-04-01                        0.4    NaN    24.1     NaN            NaN   
2005-05-01                        0.4    NaN     0.7     NaN            NaN   

               OBS_VALUE_VALUE_1000EURO                                        \
PARTNER_Labels                  Belgium France Germany Romania United Kingdom   
TIME_PERIOD                                                                     
2005-01-01                         1.09    NaN    1.02     NaN            NaN   
2005-02-01                         1.09    NaN    0.90     NaN            NaN   
2005-03-01                         1.09    NaN    0.51     NaN            NaN   
2005-04-01                         1.98    NaN    6.65     NaN            NaN   
2005-05-01                         1.09    NaN    3.01     NaN            NaN   

               OIL_PRICE $/bbl                                   \
PARTNER_Labels         Belgium     France    Germany    Romania   
TIME_PERIOD                                                       
2005-01-01           42.972278  42.972278  42.972278  42.972278   
2005-02-01           44.818211  44.818211  44.818211  44.818211   
2005-03-01           50.942879  50.942879  50.942879  50.942879   
2005-04-01           50.640476  50.640476  50.640476  50.640476   
2005-05-01           47.826573  47.826573  47.826573  47.826573   

                              agricultural_land                            \
PARTNER_Labels United Kingdom           Belgium     France        Germany   
TIME_PERIOD                                                                 
2005-01-01          42.972278           13830.0  293904.00  170310.000000   
2005-02-01          4

#### SAVE EXCEL FILE FILTERED NON STANDARDIZED

In [82]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'WHEAT_FILTERED_{now}.xlsx'
df_wheat_total_filtered.to_excel(filename, index=True)

### STANDARDIZED DATA

In [83]:
# filter the pivot table using the loc method
df_wheat_total_filtered_std = df_wheat_total_updated_std.loc[:, (slice(None), top_partners)]

# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_wheat_total_filtered_std = df_wheat_total_filtered_std.sort_index(axis=1)
df_wheat_total_filtered_std.head(5)

AVG_TAVG                                               \
PARTNER_Labels   Belgium    France   Germany   Romania United Kingdom   
TIME_PERIOD                                                             
2005-01-01     -1.407486 -1.358269 -1.169212 -1.294872      -0.907741   
2005-02-01     -1.595379 -1.709310 -1.641851 -1.687568      -1.349093   
2005-03-01     -0.966726 -0.885410 -0.905543 -1.008484      -0.789560   
2005-04-01      0.002594 -0.308336 -0.038976 -0.156987      -0.425889   
2005-05-01      0.323066  0.397419  0.467529  0.604240       0.046633   

                MAX_TMAX                                               \
PARTNER_Labels   Belgium    France   Germany   Romania United Kingdom   
TIME_PERIOD                                                             
2005-01-01     -1.733743 -1.510058 -1.103121 -1.414900      -1.039896   
2005-02-01     -1.466596 -1.570689 -1.369811 -1.940089      -1.405439   
2005-03-01     -0.321683 -0.357323 -0.470627 -0.709353      -0.554181   
2005-04-01     -0.194470  0.447180 -0.068961 -0.227240      -0.104056   
2005-05-01      0.098119  0.808253  1.186897  0.645576       0.486926   

                MIN_TMIN                                               \
PARTNER_Labels   Belgium    France   Germany   Romania United Kingdom   
TIME_PERIOD                                                             
2005-01-01     -0.972661 -1.411612 -1.450704 -1.088806      -0.872485   
2005-02-01     -1.622534 -1.858963 -1.991585 -1.964922      -0.886959   
2005-03-01     -1.288313 -2.318277 -1.583667 -1.527371      -0.950413   
2005-04-01      0.661306 -0.252798 -0.225310 -0.135713      -0.652271   
2005-05-01      0.587035  0.336368  0.321316  0.636079      -0.468973   

               OBS_VALUE_QUANTITY_TON                                          \
PARTNER_Labels                Belgium France   Germany Romania United Kingdom   
TIME_PERIOD                                                                     
2005-01-01                  -0.955531    NaN -1.183154     NaN            NaN   
2005-02-01                  -0.955531    NaN -1.183132     NaN            NaN   
2005-03-01                  -0.955531    NaN -1.183147     NaN            NaN   
2005-04-01                  -0.955531    NaN -1.182645     NaN            NaN   
2005-05-01                  -0.955531    NaN -1.183149     NaN            NaN   

               OBS_VALUE_VALUE_1000EURO                           \
PARTNER_Labels                  Belgium France   Germany Romania   
TIME_PERIOD                                                        
2005-01-01                    -1.016126    NaN -1.196589     NaN   
2005-02-01                    -1.016126    NaN -1.196602     NaN   
2005-03-01                    -1.016126    NaN -1.196643     NaN   
2005-04-01                    -1.015924    NaN -1.195987     NaN   
2005-05-01                    -1.016126    NaN -1.196376     NaN   

                              OIL_PRICE $/bbl                                \
PARTNER_Labels United Kingdom         Belgium    France   Germany   Romania   
TIME_PERIOD                                                                   
2005-01-01                NaN       -1.267942 -1.267942 -1.267942 -1.267942   
2005-02-01                NaN       -1.190552 -1.190552 -1.190552 -1.190552   
2005-03-01                NaN       -0.933780 -0.933780 -0.933780 -0.933780   
2005-04-01                NaN       -0.946458 -0.946458 -0.946458 -0.946458   
2005-05-01                NaN       -1.064429 -1.064429 -1.064429 -1.064429   

                              agricultural_land                                \
PARTNER_Labels United Kingdom           Belgium    France   Germany   Romania   
TIME_PERIOD                                                                     
2005-01-01          -1.267942          1.902207  2.327935  2.372510  2.227104   
2005-02-01          -1.190552          1.886425  2.301063  2.315924  2.168445   
2005-03-01          -0.933780

#### SAVE EXCEL FILE FILTERED STANDARDIZED

In [84]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'WHEAT_FILTERED_STANDARDIZED_{now}.xlsx'
df_wheat_total_filtered_std.to_excel(filename, index=True)